In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image 
import numpy as np
import pandas as pd
from torchvision.datasets import ImageFolder

import cv2
from sklearn.metrics import classification_report
import torch
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
import torchvision


In [2]:
data_dir = "/Users/sharanyu/Desktop/AIproject/flowers"  # Change this to the path of the dataset on your system
classes = os.listdir(data_dir)
num_classes = len(classes)
print(f"Number of classes: {num_classes}")
print(f"Classes: {classes}")


Number of classes: 16
Classes: ['calendula', 'bellflower', 'iris', 'astilbe', 'common_daisy', 'california_poppy', 'carnation', 'rose', 'tulip', 'black_eyed_susan', 'coreopsis', 'dandelion', 'water_lily', 'magnolia', 'sunflower', 'daffodil']


In [3]:
data_dir = "/Users/sharanyu/Desktop/AIproject/flowers"
class_names = os.listdir(data_dir)
num_class = len(class_names)
image_files = [[os.path.join(data_dir, class_name, x) 
               for x in os.listdir(os.path.join(data_dir, class_name))] 
               for class_name in class_names]

In [4]:
device = torch.device('mps')

In [14]:
# analyzing the dataset, determining the no. of classes, dimesnions and files/class.
rows = []
file_count = 0
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    num_files = len(os.listdir(class_dir))
    total_width = 0
    total_height = 0
    
    for img_name in os.listdir(class_dir):
        try:
          img_path = os.path.join(class_dir, img_name)
          img = cv2.imread(img_path)
          height, width, channels = img.shape
          total_width += width
          total_height += height
          file_count += 1
        except Exception as e:
          print(img_path)
          print(str(e))
    avg_width = total_width // num_files
    avg_height = total_height // num_files
    rows.append({"Class": class_name, "NumFiles": num_files, "AvgWidth": avg_width, "AvgHeight": avg_height})

df = pd.DataFrame(rows)
print(df)
print("Total files: ",file_count)

               Class  NumFiles  AvgWidth  AvgHeight
0          calendula       978       256        256
1         bellflower       873       256        256
2               iris      1054       256        256
3            astilbe       737       256        256
4       common_daisy       980       256        256
5   california_poppy      1022       256        256
6          carnation       923       256        256
7               rose       999       256        256
8              tulip      1048       256        256
9   black_eyed_susan      1000       256        256
10         coreopsis      1047       256        256
11         dandelion      1052       256        256
12        water_lily       982       781        586
13          magnolia      1048       256        256
14         sunflower      1027       256        256
15          daffodil       970       256        256
Total files:  15740


In [5]:
# Hyperparameters
num_epochs = 25
batch_size = 128
learning_rate = 0.0001

In [6]:
# Data augmentation for the train data.

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),    
    transforms.RandomRotation(10),        
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
    transforms.RandomHorizontalFlip(),  
    transforms.RandomVerticalFlip(),     
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),  
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                          [0.229, 0.224, 0.225])
])


In [7]:
dataset = ImageFolder(root='flowers', transform=train_transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# creating dataloaders with bathc size of 128
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [8]:
model = torchvision.models.resnet18(pretrained = True) # resnet transfer learning for dataset 3

for param in model.parameters():
    param.requires_grad = False

# Replace the last layer of the fully connected part of the network
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.to(device)

/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
from tqdm import tqdm

best_val_accuracy = 0.0

for epoch in range(num_epochs):
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}')):
        # Move data to device
        data = data.to(device)
        targets = targets.to(device)

        scores = model(data)
        loss = criterion(scores, targets)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predictions = torch.max(scores.data, 1)
        total += targets.size(0)
        correct += (predictions == targets).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100 * correct / total

    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}'):

            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)

            _, predictions = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predictions == targets).sum().item()

        val_accuracy = 100 * correct / total

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')

    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in tqdm(test_loader, desc=f'Test Epoch {epoch+1}/{num_epochs}'):
            data = data.to(device)
            targets = targets.to(device)

            scores = model(data)

            _, predictions = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predictions == targets).sum().item()

        test_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Accuracy: {val_accuracy:.2f}%, '
          f'Test Accuracy: {test_accuracy:.2f}%')


Test Epoch 1/25: 100%|██████████| 25/25 [00:19<00:00,  1.29it/s]


Epoch 1/25, Train Loss: 2.7542, Train Accuracy: 10.83%, Val Accuracy: 17.15%, Test Accuracy: 16.39%


Test Epoch 2/25: 100%|██████████| 25/25 [00:20<00:00,  1.24it/s]


Epoch 2/25, Train Loss: 2.4649, Train Accuracy: 25.51%, Val Accuracy: 33.48%, Test Accuracy: 32.66%


Test Epoch 3/25: 100%|██████████| 25/25 [00:18<00:00,  1.35it/s]


...

Epoch 23/25, Train Loss: 1.0607, Train Accuracy: 69.99%, Val Accuracy: 69.25%, Test Accuracy: 69.79%


Test Epoch 24/25: 100%|██████████| 25/25 [00:19<00:00,  1.31it/s]


Epoch 24/25, Train Loss: 1.0477, Train Accuracy: 70.25%, Val Accuracy: 69.06%, Test Accuracy: 69.09%


Test Epoch 25/25: 100%|██████████| 25/25 [00:19<00:00,  1.26it/s]

Epoch 25/25, Train Loss: 1.0401, Train Accuracy: 70.38%, Val Accuracy: 70.14%, Test Accuracy: 70.01%


In [10]:
#  initial run fullOutput:

'''    Epoch 1/25, Batch 0/79, Loss 2.9198
    Validation accuracy: 19.61%
    Test accuracy: 19.12%
    Epoch 2/25, Batch 0/79, Loss 2.5499
    Validation accuracy: 34.10%
    Test accuracy: 33.04%
    Epoch 3/25, Batch 0/79, Loss 2.3307
    Validation accuracy: 44.54%
    Test accuracy: 44.35%
    Epoch 4/25, Batch 0/79, Loss 2.1449
    Validation accuracy: 51.13%
    Test accuracy: 50.73%
    Epoch 5/25, Batch 0/79, Loss 2.0068
    Validation accuracy: 55.22%
    Test accuracy: 54.67%
    Epoch 6/25, Batch 0/79, Loss 1.8410
    Validation accuracy: 58.67%
    Test accuracy: 58.13%
    Epoch 7/25, Batch 0/79, Loss 1.6271
    Validation accuracy: 60.06%
    Test accuracy: 59.91%
    Epoch 8/25, Batch 0/79, Loss 1.5167
    Validation accuracy: 61.93%
    Test accuracy: 60.99%
    Epoch 9/25, Batch 0/79, Loss 1.5282
    Validation accuracy: 62.76%
    Test accuracy: 62.23%
    Epoch 10/25, Batch 0/79, Loss 1.3853
    Validation accuracy: 63.80%
    Test accuracy: 63.63%
    Epoch 11/25, Batch 0/79, Loss 1.4738
    Validation accuracy: 65.11%
    Test accuracy: 63.95%
    Epoch 12/25, Batch 0/79, Loss 1.3234
    Validation accuracy: 66.14%
    Test accuracy: 65.41%
    Epoch 13/25, Batch 0/79, Loss 1.3975
    Validation accuracy: 66.42%
    Test accuracy: 65.47%
    Epoch 14/25, Batch 0/79, Loss 1.3211
    Validation accuracy: 66.38%
    Test accuracy: 66.26%
    Epoch 15/25, Batch 0/79, Loss 1.2833
    Validation accuracy: 66.61%
    Test accuracy: 66.26%
    Epoch 16/25, Batch 0/79, Loss 1.2273
    Validation accuracy: 67.29%
    Test accuracy: 67.09%
    Epoch 17/25, Batch 0/79, Loss 1.1177
    Validation accuracy: 66.97%
    Test accuracy: 67.34%
    Epoch 18/25, Batch 0/79, Loss 1.0788
    Validation accuracy: 67.69%
    Test accuracy: 68.04%
    Epoch 19/25, Batch 0/79, Loss 1.2609
    Validation accuracy: 67.84%
    Test accuracy: 67.79%
    Epoch 20/25, Batch 0/79, Loss 1.0463
    Validation accuracy: 68.40%
    Test accuracy: 68.30%
    Epoch 21/25, Batch 0/79, Loss 1.0968
    Validation accuracy: 68.44%
    Test accuracy: 68.65%
    Epoch 22/25, Batch 0/79, Loss 1.0011
    Validation accuracy: 68.52%
    Test accuracy: 68.87%
    Epoch 23/25, Batch 0/79, Loss 1.3195
    Validation accuracy: 68.72%
    Test accuracy: 69.12%
    Epoch 24/25, Batch 0/79, Loss 0.9101
    Validation accuracy: 68.80%
    Test accuracy: 69.22%
    Epoch 25/25, Batch 0/79, Loss 0.9813
    Validation accuracy: 69.08%
    Test accuracy: 69.47%
'''

'    Epoch 1/25, Batch 0/79, Loss 2.9198\n    Validation accuracy: 19.61%\n    Test accuracy: 19.12%\n    Epoch 2/25, Batch 0/79, Loss 2.5499\n    Validation accuracy: 34.10%\n    Test accuracy: 33.04%\n    Epoch 3/25, Batch 0/79, Loss 2.3307\n    Validation accuracy: 44.54%\n    Test accuracy: 44.35%\n    Epoch 4/25, Batch 0/79, Loss 2.1449\n    Validation accuracy: 51.13%\n    Test accuracy: 50.73%\n    Epoch 5/25, Batch 0/79, Loss 2.0068\n    Validation accuracy: 55.22%\n    Test accuracy: 54.67%\n    Epoch 6/25, Batch 0/79, Loss 1.8410\n    Validation accuracy: 58.67%\n    Test accuracy: 58.13%\n    Epoch 7/25, Batch 0/79, Loss 1.6271\n    Validation accuracy: 60.06%\n    Test accuracy: 59.91%\n    Epoch 8/25, Batch 0/79, Loss 1.5167\n    Validation accuracy: 61.93%\n    Test accuracy: 60.99%\n    Epoch 9/25, Batch 0/79, Loss 1.5282\n    Validation accuracy: 62.76%\n    Test accuracy: 62.23%\n    Epoch 10/25, Batch 0/79, Loss 1.3853\n    Validation accuracy: 63.80%\n    Test accura

70.1397712833545